In [3]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

In [113]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        
    def guess(self, word): # word input example: "_ p p _ e "
       
        # create a string to store guessed lettes
        guessed_letters_str = "".join(self.guessed_letters)

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".")
        
        # find length of passed word
        len_word = len(clean_word)
        
        # create a pattern to match the possible words
        # for the input word that has no letters, no need to exclude any letters
        # for a pattern such as .pp.e with tried letters: e,p,a
        if guessed_letters_str != '':
            # match the words that excludes tried letters (i.e., e,p,a)
            regex = clean_word.replace('.','[^'+guessed_letters_str+']')
        
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        # create the list that stores the unique letter per word
        uniq_dictionary = []
        # iterate through all of the words in the old plausible dictionary
        if re.search('[a-zA-Z]', clean_word):
            # if dictionary word is a possible match then add it to the current dictionary
            for dict_word in current_dictionary:
                if re.match(regex,dict_word):
                    new_dictionary.append(dict_word)
                    # use set to guarantee each letter per word counted once
                    uniq_dictionary.append(set(dict_word))
        else:
            for dict_word in current_dictionary:
                # if input word contains no letters, it's enough to find possible words with the same length
                if len(dict_word) == len_word:
                    new_dictionary.append(dict_word)
                    # make sure we're only counting a letter once per word
                    uniq_dictionary.append(set(dict_word))
                
            
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        
        
        # convert the list of unique letters per word to string
        letters = sorted("".join(str(v) for v in uniq_dictionary))
        letters = " ".join(str(x) for x in letters)
        
        # remove the non-English characters, i.e., {, "
        letters = re.sub(r'[^a-zA-Z]', "", letters)
        
        # count occurrence of all characters in possible word matches 
        c = collections.Counter(letters)
        
        # sort the occurence of letters 
        sorted_letter_count = c.most_common() 
        
        guess_letter = '!'
        full_dictionary = self.full_dictionary
        
        # create a list to store words that haven't been guessed
        left_dic=[]
        
        # create a list to store words that haven't been guessed
        left_uniq=[]
        
        # begin guessing words
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break
            
        
            
        # if no word matches in training dictionary, default back to ordering of full dictionary that excludes the words
        # in the current dictionay
        
        if guess_letter == '!':
            
            # sort the occurences of letter in the words that weren't in the current dictionary
            current_dictionary = [x for x in full_dictionary if x not in current_dictionary]
            for dict_word in current_dictionary:
                    # make sure we're only counting a letter once per word
                left_uniq.append(set(dict_word))
                
            # convert the list of counting unique letters per word to string
            left_letters = sorted("".join(str(v) for v in left_uniq))
            left_letters = " ".join(str(x) for x in left_letters)
            
            # only keep the English letters in the string
            left_letters = re.sub(r'[^a-zA-Z]', "", left_letters)
    
            # sort the occurences of letters 
            left_c = collections.Counter(left_letters)
            sorted_letter_count = left_c.most_common()
            
            # begin guessing
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break            
        
        return guess_letter

    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        try:
            # response = self.session.request(
            response = requests.request(
                method or "GET",
                HANGMAN_URL + path,
                timeout=self.timeout,
                params=args,
                data=post_args)
        except requests.HTTPError as e:
            response = json.loads(e.read())
            raise HangmanAPIError(response)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)